In [1]:
import numpy as np
import os
import scipy.io
from scipy import signal
from scipy.integrate import simps

In [2]:
def loadData(folderName):
    data = []
    labels = []
    allFiles = os.listdir(folderName)
    for file in allFiles:
        loadedFile = scipy.io.loadmat(folderName+'/'+file)
        data.append(loadedFile['data'])
        labels.append(loadedFile['labels'])

    return np.array(data), np.array(labels)

In [3]:
data, labels = loadData("../Datasets/data_preprocessed_matlab")

In [4]:
# subject * video * channel * data
print(data.shape)
# subject * video * labels
print(labels.shape)

(32, 40, 40, 8064)
(32, 40, 4)


In [5]:
def getSpecificChannels():
    channels = np.array(['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz',
                        'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2'])
    selectedChannel = np.array(
        ['AF3', 'F3', 'F7', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F8', 'F4', 'AF4'])
    return np.where(np.in1d(channels, selectedChannel))[0]

In [6]:
channelIndices = getSpecificChannels()
channelIndices

array([ 1,  2,  3,  4,  7, 11, 13, 17, 19, 20, 21, 25, 29, 31])

In [7]:
# Select 14 channel
selectedChannelData = data[:,:,channelIndices,:]
print(selectedChannelData.shape)

(32, 40, 14, 8064)


In [8]:
# Remove first 20 second of video
sampleRate = 128
removed20SecData = selectedChannelData[:,:,:,sampleRate*20:sampleRate*60]

In [9]:
removed20SecData.shape

(32, 40, 14, 5120)

In [10]:
processedData = np.zeros((32,160,14,1280))
parts = [removed20SecData[:,:,:,:sampleRate*10],
         removed20SecData[:,:,:,sampleRate*10:sampleRate*20],
         removed20SecData[:,:,:,sampleRate*20:sampleRate*30],
         removed20SecData[:,:,:,sampleRate*30:]
        ]

for s in range(32):
    for v in range(40):
        for i in range(4):
            processedData[s,(v * 4)+i, :,:] = parts[i][s,v,:,:]
            
processedLabels = np.repeat(labels,4, axis=1)

In [11]:
print(processedData.shape)
print(processedLabels.shape)

(32, 160, 14, 1280)
(32, 160, 4)


In [12]:
valenceLabels = ((processedLabels[:, :, 0] >= 5) * 1)
arousalLabels = ((processedLabels[:, :, 1] >= 5) * 1)
dominanceLabels = ((processedLabels[:, :, 2] >= 5) * 1)

In [13]:
subjectCount = 32
channelCount = 14
videoCount = 160
bands = [
    {
        'low':4,
        'high':8
    },
    {
        'low':9,
        'high':12
    },{
        'low':13,
        'high':30,
    },{
        'low':31,
        'high':45
}]
asymetryChannels = np.array([
    [0,7],
    [1,8],
    [2,9],
    [4,10],
    [4,11],
    [5,12],
    [6,13]
])

In [14]:
def bandPowerFeature(data,sampleRate):
    win = 10 * sampleRate
    bandPower = []
    for band in bands:
        freqs, psd = signal.welch(data, sampleRate, nperseg=win)
        freq_res = freqs[1] - freqs[0]
        idx = np.logical_and(freqs >= band['low'], freqs <= band['high'])
        power = simps(psd[idx], dx=freq_res)
        bandPower.append(power)
    return np.array(bandPower)

In [15]:
def extractPowerFeatures(data):
    sampleRate = 128
    subjectFeatures = []
    for s in range(subjectCount):
        
        videoFeatures = []
        for v in range(videoCount):
            
            channelFeatures = []
            for ch in range(channelCount):
                item = data[s][v][ch]
                powerFeature = bandPowerFeature(item,sampleRate)
                channelFeatures.append(powerFeature)
            
            videoFeatures.append(channelFeatures)
        subjectFeatures.append(videoFeatures)
    return np.array(subjectFeatures)

In [16]:
def bandDASMFeature(data):
    indices = asymetryChannels
    sampleRate = 128
        
    subjectFeatures = []
    for s in range(subjectCount):
        
        videoFeatures = []
        for v in range(videoCount):
            
            channelFeatures = []
            for index in indices:
                firstChannelPower = bandPowerFeature(data[s][v][index[0]],sampleRate)
                secChannelPower = bandPowerFeature(data[s][v][index[1]],sampleRate)
                channelFeatures.append(firstChannelPower - secChannelPower)
                
            videoFeatures.append(channelFeatures)
        subjectFeatures.append(videoFeatures)
    return np.array(subjectFeatures)

In [17]:
def bandRASMFeature(data):
    indices = asymetryChannels
    sampleRate = 128
        
    subjectFeatures = []
    for s in range(subjectCount):
        
        videoFeatures = []
        for v in range(videoCount):
            
            channelFeatures = []
            for index in indices:
                firstChannelPower = bandPowerFeature(data[s][v][index[0]],sampleRate)
                secChannelPower = bandPowerFeature(data[s][v][index[1]],sampleRate)
                channelFeatures.append(firstChannelPower / secChannelPower)
                
            videoFeatures.append(channelFeatures)
        subjectFeatures.append(videoFeatures)
    return np.array(subjectFeatures)

In [18]:
def crossPowerFeature(s1,s2, sampleRate):
    win = 10 * sampleRate
    crossPower = []
    for band in bands:
        freqs, csd = signal.csd(s1,s2, sampleRate, nperseg=win)
        freq_res = freqs[1] - freqs[0]
        idx = np.logical_and(freqs >= band['low'], freqs <= band['high'])
        power = simps(csd[idx], dx=freq_res)
        crossPower.append(power.real)
        crossPower.append(power.imag)
    return crossPower

In [19]:
def extractCrossPowerFeature(data):
    indices = asymetryChannels
    sampleRate =128
    
    subjectCrossPower = []
    for s in range(subjectCount):
        
        videoFeatures = []
        for v in range(videoCount):
        
            channelFeatures=[]
            for index in indices:
                crossFeatuer = crossPowerFeature(data[s][v][index[0]],data[s][v][index[1]],sampleRate)
                channelFeatures.append(crossFeatuer)
            
            videoFeatures.append(channelFeatures)
            
        subjectCrossPower.append(videoFeatures)
    return np.array(subjectCrossPower)

In [28]:
powerFeatures = extractPowerFeatures(processedData)
print(powerFeatures.shape)

(32, 160, 14, 4)


In [22]:
dasmFeatures = bandDASMFeature(processedData)
print(dasmFeatures.shape)

(32, 160, 7, 4)


In [23]:
rasmFeatures = bandRASMFeature(processedData)
print(rasmFeatures.shape)

(32, 160, 7, 4)


In [26]:
crossFeatures = extractCrossPowerFeature(processedData)
print(crossFeatures.shape)

(32, 160, 7, 8)


In [29]:
powerFeatures = powerFeatures.reshape(powerFeatures.shape[0],powerFeatures.shape[1],powerFeatures.shape[2]*powerFeatures.shape[3])
dasmFeatures = dasmFeatures.reshape(dasmFeatures.shape[0],dasmFeatures.shape[1],dasmFeatures.shape[2]*dasmFeatures.shape[3])
rasmFeatures = rasmFeatures.reshape(rasmFeatures.shape[0],rasmFeatures.shape[1],rasmFeatures.shape[2]*rasmFeatures.shape[3])
crossFeatures = crossFeatures.reshape(crossFeatures.shape[0],crossFeatures.shape[1],crossFeatures.shape[2]*crossFeatures.shape[3])

In [32]:
features = np.dstack((powerFeatures,crossFeatures,dasmFeatures,rasmFeatures))
print(features.shape)

(32, 160, 168)


In [36]:
import pickle
powerFeaturesFolder = 'Dataset/bandPowerFeatures.dat'
with open(powerFeaturesFolder,'wb') as f:
    pickle.dump(features,f)

In [37]:
with open(powerFeaturesFolder,'rb') as f:
    bandPowers = pickle.load(f)

In [38]:
print(bandPowers.shape)

(32, 160, 168)
